# Aldebaran Spectra historical review
This script will query from VizieR, spectra analysis for the star Aldebaran at cooridinates 04+35+55.23907 16+30+33.4885. Use via the interactive plot in this notebook after uploading it to google collab, open and run all cells. Collab will prompt you download necessary packages.

In [ ]:
# Necessary Imported Packages
!pip install astroquery
!pip install plotly
!pip install astropy
import numpy
import numpy
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from astroquery.vizier import Vizier
from datetime import datetime
import plotly.express as px


# Setup
Read table from VizieR's Photometry Webpage: (http://vizier.cds.unistra.fr/vizier/sed/)

In [10]:
radius=2 # Radius in Arcseconds 
target="04+35+55.23907%2B16+30+33.4885" # Url Encoded Coordiates
sed=Table.read(f"https://vizier.cds.unistra.fr/viz-bin/sed?-c={target}&-c.rs={radius}") # Read Spectra table from vizieR photometry


Cross the catalog descriptions for their year of publish and add these to a pandas dataframe.

In [15]:
df=sed.to_pandas() # Convert table to Pandas dataframe
names=df['_tabname'].values # Grab the catalog names from the dataframe
outputs=[] # Define outputs array
for n in names: # Use astroquery VizieR package to find the metadata description of the colleagues.
    catalog_list = Vizier.find_catalogs(n)
    for k, v in catalog_list.items():
       outputs.append(v.description)
# Grabs a description in the format:
# The Hipparcos and Tycho Catalogues (ESA 1997) The Hipparcos Main Catalogue
import re #Use re.search to extract the year from this to ascribe a time to the measurement.
years=[] # output years array
c=0
for o in outputs:
    finds=re.findall(r'(.*?)(\([A-Za-z][A-Za-z\'\s\-\/]*[\+,]?(?:\s*et al\.)?,?\s(?:\d{4}(?:\-\d{0,4})?|\d{4}\-\d{2}\-\d{2}|Version\s[\d\-]+)\))(.*)',o) # Extracts the citation text from *generally* closed parenthesis.
    # Will be left with ('The Hipparcos and Tycho Catalogues ', '(ESA 1997)', '')
    # Filter out the none citation text, this will look at the first instance of a date that makes sense(ie before 2026)
    for f in list(finds[0]):
        try:
            year_found=int(re.findall(r'\d{4}',f)[0])  #Trys to match in filtered strings, passing if it doesn't find a valid year
            #print(year_found)
            if year_found<2026: # selects the first good year
                years.append(str(year_found)) # Appending to found year, assigning to string for plotly
                break
            else:
                pass
        except:
            pass

### Future Implementation
If we have data from a specific month or year. We can instead extract the month, day or second from the timedate info given.

In [16]:
df['time']=years # Assign the time bin for data sorting.
c=3*10**8
df['wavelength']=(c/(df['sed_freq']*10**6))/10**-9 #converts frequecny to wavelength in nanometers
df.sort_values('time')

,_RAJ2000,_DEJ2000,_tabname,_ID,_tab1_86,_tab1_87,sed_freq,sed_flux,sed_eflux,sed_filter,time,wavelength
104,68.980163,16.509302,III/126/part1,recno=1411,NaN,0.0,541430.0,1650.00,NaN,Johnson:V,1588,5.540882e+05
105,68.980163,16.509302,III/126/part2,recno=456,NaN,0.0,541430.0,1650.00,NaN,Johnson:V,1588,5.540882e+05
106,68.980163,16.509302,III/126/part3,recno=411,NaN,0.0,541430.0,1650.00,NaN,Johnson:V,1588,5.540882e+05
107,68.980163,16.509302,III/126/part3,recno=412,NaN,0.0,541430.0,1650.00,NaN,Johnson:V,1588,5.540882e+05
65,68.980163,16.509303,II/122B/merged,recno=22866,NaN,0.0,541430.0,1650.00,NaN,Johnson:V,1953,5.540882e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
350,68.980246,16.508900,J/A+A/688/A203/bsrejcds,recno=11321,NaN,0.0,4282.8,14.30,0.00,Herschel/PACS:70,2024,7.004763e+07
351,68.980235,16.508904,J/A+A/688/A203/rrejcds,recno=34099,NaN,0.0,1873.7,2.84,0.06,Herschel/PACS:160,2024,1.601110e+08
352,68.980458,16.508848,J/A+A/688/A203/rrejcds,recno=34100,NaN,0.0,1873.7,2.82,0.05,Herschel/PACS:160,2024,1.601110e+08
347,68.980000,16.509311,J/A+A/681/A107/metaspec,recno=132,2.455577e+06,1100.0,541430.0,1650.00,NaN,Johnson:V,2024,5.540882e+05


Utilize Plotly to show data collected by year, click the values on the right to hide specific years. While not perfect, generally 1 study is done a year. Utilize the button to hide all the years or show all. There is are viewing options provided by plotly for you to look at if you're interested. The flux is stored in the table in Janskys, while the wavelength is stored as frequency($f=c/\lambda$)

In [ ]:
fig=px.scatter(df.sort_values('time'),x="wavelength",y="sed_flux",title="Emitting Spectra of Aldebaran(Alpha Tauri)",log_x=True,color="time",labels={"wavelength": "Wavelength(nm)", "sed_flux": "Flux(Jy)"}) # Defines plotly scatter plot with x axis in logarithmic
fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
fig.show()